<a href="https://colab.research.google.com/github/Priyo-prog/Statistics-and-Data-Science/blob/main/Feature%20Selection%20Complete/Wrapper%20Method/wrapper_step_forward.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Step forward feature selection**
Step forward feature selection starts by training a machine learning model for each feature in the dataset and selecting, as the starting feature, the one that returns the best performing model according to the evaluation criteria we choose.

In the second step, it creates machine learning models for all combinations of the feature selected in the previous step and a second feature. It selects the pair that produces the best performing algorithm.

It continues by adding 1 feature at a time to the features that were selected in previous steps, until a stopping criteria is reached.

In theory, models with more features perform better. The algorithm will continue adding new features until a certain criteria is met. For example, until the model performance does not increase beyond a certain threshold. Or, as we show in this notebook, until a certain number of features are selected.

The model performance metric can be the roc_auc for classification and the r squared for regression, for example, and it is determined by the user.

Step forward feature selection is called a greedy procedure because it evaluates many possible single, double, triple, and so on feature combinations. Therefore, it is very computationally expensive and, sometimes, if the feature space is big enough, even unfeasible.

There is a special package in Python that implements this type of feature selection: mlxtend.

In the mlxtend implementation of the Step Forward Feature Selection, the original stopping criteria is an arbitrarily set number of features. So the search will finish when we reach the desired number of selected features.

This is somewhat arbitrary, we might be selecting a sub-optimal number of features, or likewise, a high number of features. But, by looking at the performance metric returned by the algorithm as it selects the features, we can have a view on whether more features do add value, or not.

**Note**

In latest releases, MLXtend incorporated alternative stopping criterias. Visit the documentation for more details.

Here I will use the Step Forward feature selection algorithm from MLXtend in a classification and regression dataset.

In [11]:
# Import important libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, r2_score

from sklearn.feature_selection import SequentialFeatureSelector as SFS

In [2]:
# Mount the Google Drive
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
# import the dataset
filename = "/content/drive/MyDrive/Data Science/Feature Selection/dataset_2.csv"

In [5]:
df = pd.read_csv(filename)
df.head()

,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,...,var_100,var_101,var_102,var_103,var_104,var_105,var_106,var_107,var_108,var_109
0,4.532710,3.280834,17.982476,4.404259,2.349910,0.603264,2.784655,0.323146,12.009691,0.139346,...,2.079066,6.748819,2.941445,18.360496,17.726613,7.774031,1.473441,1.973832,0.976806,2.541417
1,5.821374,12.098722,13.309151,4.125599,1.045386,1.832035,1.833494,0.709090,8.652883,0.102757,...,2.479789,7.795290,3.557890,17.383378,15.193423,8.263673,1.878108,0.567939,1.018818,1.416433
2,1.938776,7.952752,0.972671,3.459267,1.935782,0.621463,2.338139,0.344948,9.937850,11.691283,...,1.861487,6.130886,3.401064,15.850471,14.620599,6.849776,1.098210,1.959183,1.575493,1.857893
3,6.020690,9.900544,17.869637,4.366715,1.973693,2.026012,2.853025,0.674847,11.816859,0.011151,...,1.340944,7.240058,2.417235,15.194609,13.553772,7.229971,0.835158,2.234482,0.946170,2.700606
4,3.909506,10.576516,0.934191,3.419572,1.871438,3.340811,1.868282,0.439865,13.585620,1.153366,...,2.738095,6.565509,4.341414,15.893832,11.929787,6.954033,1.853364,0.511027,2.599562,0.811364


**Important**

In all feature selection procedures, it is good practice to select the features by examining only the training set. And this is to avoid overfit.

In [6]:
# Separate the features and labels
X = df.drop(labels=["target"], axis=1)
y = df["target"]

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=1)
X_train.shape, X_test.shape

((35000, 108), (15000, 108))

**Remove Correlated features**

Step Forward Feature Selection takes a long time to run, so to speed it up we will reduce the feature space by removing correlated features first.

In [8]:
# Remove correlated features to reduce the feature space

def correlation(dataset, threshold):

  col_corr = set()

  corr_matrix = dataset.corr()

  for i in range(len(corr_matrix.columns)):
    for j in range(i):
      if abs(corr_matrix.iloc[i,j]) > threshold:
        colname = corr_matrix.columns[i]
        col_corr.add(colname)

  return col_corr

In [9]:
corr_features = correlation(X_train, 0.8)
print("correlated Features: ", len(set(corr_features)))

correlated Features:  36


In [10]:
# Remove the correlated features
X_train = X_train.drop(labels=corr_features, axis=1)
X_test.drop(labels=corr_features, axis=1, inplace=True)

**Step Forward Feature Selection**

For the Step Forward feature selection algorithm, we are going to use the class SFS from Sklearn

In [13]:
rf = RandomForestClassifier(n_estimators=10, n_jobs=-1, random_state=0)

sfs1 = SFS(estimator=rf,
           n_features_to_select=10,
           tol=None,
           direction="forward",
           scoring="roc_auc",
           cv=2,
           n_jobs=-1
)

In [14]:
sfs1.fit(X_train,y_train)

SequentialFeatureSelector(cv=2,
                          estimator=RandomForestClassifier(n_estimators=10,
                                                           n_jobs=-1,
                                                           random_state=0),
                          n_features_to_select=10, n_jobs=-1,
                          scoring='roc_auc')

In [16]:
selected_features = sfs1.get_feature_names_out()
selected_features

array(['var_13', 'var_14', 'var_16', 'var_21', 'var_41', 'var_45',
       'var_55', 'var_69', 'var_91', 'var_98'], dtype=object)